In [6]:
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import YoutubeLoader
import streamlit as st
import os

In [4]:
OPENAI_API_KEY = st.secrets["OPENAI_API_KEY"]
youtube_url = "https://www.youtube.com/watch?v=P1iCMyuwOfg&ab_channel=ChefReactions"

In [7]:
loader = YoutubeLoader.from_youtube_url(youtube_url)

In [13]:
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model = "text-embedding-ada-002")
vector_store = FAISS.from_documents(chunks, embedding)

In [11]:
len(chunks)

2